Alicia Sigmon

als333@pitt.edu

11/19/2017

**NOTE: This script is an EXISTING script, meaning that I am continuously adding to and modifying this script.**

# Discourse Analysis of the Australian Radio Talkback Corpus

#### About the Corpus
- The Australian Radio Talkback corpus contains raw text files of telephone conversations.
- These conversations include the speaker's role (presenter, expert, or caller), name, and gender.
- The conversations include other verbal cues such as laughter <laugh> and where a speaker says something during another speaker's turn <E1 yeah>. It also notes corrections to the transcription in squirrley brackets {}.

#### Discourse Analysis Goals
- commparing speakers by role and gender
- aspects to consider:
    - back channels (yeah, uhhu, that's great!)
    - vocabulary size (avg word length)
    - number of turns, sentences, and words

In [1]:
%pprint

Pretty printing has been turned OFF


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from nltk.corpus import PlaintextCorpusReader
import pandas as pd
import nltk
import numpy as np
import glob

In [4]:
ART_fids=glob.glob('C:\\Users\\sigmo\\Documents\\Data_Science\\Discourse-Analysis-ART-Corpus\\data_files\\AustralianRadioTalkback\\files\\Raw\\*.txt')

**Each filename contains 3-5 letters followed by a number. They all end in -raw.txt.**

    ex: ABCE1-raw.txt, NAT4-raw.txt
    
## Getting the Files

In [5]:
# having trouble with glob..

print("There are " + str(len(ART_fids)) + " files in this corpus:") # 26
for x in ART_fids: 
#     x.replace('C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_files\AustralianRadioTalkback\files\Raw\',"")
    print(x) # I want to use .replace() to get rid of the excess, but I'm not sure how.

There are 26 files in this corpus:
C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_files\AustralianRadioTalkback\files\Raw\ABCE1-raw.txt
C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_files\AustralianRadioTalkback\files\Raw\ABCE2-raw.txt
C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_files\AustralianRadioTalkback\files\Raw\ABCE3-raw.txt
C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_files\AustralianRadioTalkback\files\Raw\ABCE4-raw.txt
C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_files\AustralianRadioTalkback\files\Raw\ABCNE1-raw.txt
C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_files\AustralianRadioTalkback\files\Raw\ABCNE2-raw.txt
C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_files\AustralianRadioTalkback\files\Raw\COME1-raw.txt
C:\Users\sigmo\Documents\Data_Science\Discourse-Analysis-ART-Corpus\data_f

In [6]:
dir(ART_fids)

['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']

## Getting the Texts

In [7]:
# Creating a dictionary of each files' text
    # keys = filenames
    
rawtext_dict = {}
for fid in ART_fids:     
    f=open(fid, 'r')  
    last_slash=fid.rindex('\\') # the key will be only the file name, not the entire location
    rawtext_dict[fid[last_slash+1:]] = f.read()
    f.close() 

In [8]:
rawtext_dict['ABCE1-raw.txt'][:1000]
# Within this glimpse of the data, the speaker information data format is shown through Presenter 1.

"[Presenter 1: Simon Marnie, M] Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation {propagation} Mr propagation. He's also known for his passion for natives and his love of o orchids am I right so far.\n\n[Expert 1: Angus Stewart, M] I guess yeah yeah <laughs>.\n\n[P1] He's also known <E1 sounds reasonable> for his ability to open cosposting {composting} toilets so he can tell you anything worm farm problems certainly helped us and although I'm still confused about dry ingredients we might talk about that as well but eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two fine sunny day today top temperatures on the coast of twenty-seven inland thirty degrees Bowral enjoying

### Original Speaker Data Format in the Raw Text Files:

   *There are multiple exceptions to this format that I will address later in the script.*

All speaker information is contained within [ ].

Each first instance of a speaker is in the following order:
    - Speaker Type
        - Presenter, Caller, or Expert
    - Speaker Number
        - the first presenter in a file is Presenter 1, the second presenter is Presenter 2, etc..
    - The Speaker's Name
        - ex: Simon Marnie
    - The Speaker's gender:
        - M / F
    
For every other instance of a speaker's line, the speaker is indicated by P, C, or E followed by their number.
    - Ex: [Presenter 1: Simon Marnie, M] == [P1]

In [9]:
print(rawtext_dict['NAT4-raw.txt']) # both NAT4-raw.txt and NAT5-raw.txt are contained within this file
# also, Caller 33b: Chris, male <-- fix gender

[Presenter 1: Tony Delroy, M] Eighteen minutes past ten eighteen past nine in Queensland eighteen past seven in Western Australia. Navigating the world of treatment for menopause can be a nightmare. In the last month we've seen the cancellation of a seven year trial in the states <,> sending more shockwaves through the community. Already a topic that causes a lotta confusion hormone replacement therapy or H R T has been controversial since its widespread use in the early nineties but does that mean it's not beneficial for some people. Menopause is different for each woman <,> uh some women suffer few side effects during the change as it's affectionately known and uh others suffer severe weight loss loss of sex drive uh mood swings and of course the dreaded hot flushes. So uh how do we make sense of everything that's on offer. To help us out tonight Dr Barry Wren is a guest. S one of Australia's leading researchers in gynaecological research and the author of a book called Understanding

### Splitting NAT4-raw.txt into 2 files: NAT4-raw.txt and NAT5-raw.txt

In [10]:
# Modifying the text keys to separate the two radio talk shows from within NAT4-raw.txt to NAT4-raw.txt and NAT5-raw.txt

seg2_start = rawtext_dict['NAT4-raw.txt'].rindex('[Presenter 1')   # Location of last mention of Presenter 1
seg1 = rawtext_dict['NAT4-raw.txt'][:seg2_start]
seg2 = rawtext_dict['NAT4-raw.txt'][seg2_start:]
seg2[:500]

rawtext_dict['NAT4-raw.txt'] = seg1
rawtext_dict['NAT5-raw.txt'] = seg2

files = sorted(rawtext_dict.keys()) # creates an accurate file list (ART_fids is missing NAT5-raw.txt!!)

"[Presenter 1: John Cleary, M] So now it's welcome first to our expert panel. Dr Brian Edgar he is director of theology and public policy for the Evangelical Alliance a mainstream Protestant agency which have a website that canvases electoral issues. Brian welcome <Expert 1: Brian Edgar, M thank you John> to you.\n\n[E1] Thank you very much glad to <P1 Victoria> be here.\n\n[P1] Victoria Kearney is one of the coordinators of a website called PolMin which looks at lobbying for policies in harmony with"

### Trial Run: Splitting Texts by Lines

In [11]:
# How to split lines using regular expressions 
    # some lines appeared to contain \r while most contained \n
import re
foo = "Hello world\n\nhow are you\na new line\r\nanother newline\n"
re.split(r'[\n\r]+', foo)
re.split(r'[\n\r]+', foo.strip())

['Hello world', 'how are you', 'a new line', 'another newline', '']

['Hello world', 'how are you', 'a new line', 'another newline']

In [12]:
# Splitting the texts by line:
    # Trial run on the first 3 files and their first 20 lines:

for fid in files[:3]: # files includes  NAT5-raw.txt
    rawtext = rawtext_dict[fid] # already includes NAT5-raw.txt
    rawlines = re.split(r'[\n\r]+', rawtext.strip())[:20]
    print(fid)
    for l in rawlines:
        if ']' in l:
            where = l.index(']')
            speaker = l[:where+1]
            utterance = l[where+2:]
            print(speaker+' '+utterance)
        else: 
            print('***'+l+'******')    # {program advert}. What to do with these? 
    print()

ABCE1-raw.txt
[Presenter 1: Simon Marnie, M] Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation {propagation} Mr propagation. He's also known for his passion for natives and his love of o orchids am I right so far.
[Expert 1: Angus Stewart, M] I guess yeah yeah <laughs>.
[P1] He's also known <E1 sounds reasonable> for his ability to open cosposting {composting} toilets so he can tell you anything worm farm problems certainly helped us and although I'm still confused about dry ingredients we might talk about that as well but eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two fine sunny day today top temperatures on the coast of twenty-seven inland thirty degrees Bowral e

## Data Cleaning: Fixing Formatting Errors

In [13]:
# Data Cleaning:

rawtext_dict['ABCE4-raw.txt'] = rawtext_dict['ABCE4-raw.txt'].replace('[E2]', '[E1]')

# There is no E2 in this segment, but E2 is erroneously mentioned. Replacing. 

# $ grep Expert  ABCE4-raw.txt 
# [Expert 1: Ric Nattrass, M] Uh blue-tongues'd be {break} unlikely ...

# $ grep E2  ABCE4-raw.txt 
# [E2] Yeah.
# [E2] Yeah okay so your yours up there is the spotted catbird if you're on <C3 mm> ...


rawtext_dict['COME2-raw.txt'] = rawtext_dict['COME2-raw.txt'].replace('[C5: Jenny, F]', '[Caller 5: Jenny, F]')


# $ grep "C5" COME2-raw.txt 
# [C5: Jenny, F] Hello how are you.
# [C5] That's good. Um I was just wondering for some information on my house at ...
# [C5] Oh have I.

rawtext_dict['ABCE3-raw.txt'] = rawtext_dict['ABCE3-raw.txt'].replace('[Caller 11, Robyn, F]', '[Caller 11: Robyn, F]')
rawtext_dict['COME1-raw.txt'] = rawtext_dict['COME1-raw.txt'].replace('[Caller 23, Maureen, F]', '[Caller 23: Maureen, F]')
rawtext_dict['NAT7-raw.txt'] = rawtext_dict['NAT7-raw.txt'].replace('[Caller 12, Brian, M]', '[Caller 12: Brian, M]')
rawtext_dict['NAT8-raw.txt'] = rawtext_dict['NAT8-raw.txt'].replace('[Caller 10, Brett, M]', '[Caller 10: Brett, M]')

rawtext_dict['NAT4-raw.txt'] = rawtext_dict['NAT4-raw.txt'].replace('[Caller 33b: Chris, male]', '[Caller 33b: Chris, M]')


# $ grep -P 'Caller \d+,' *
# ABCE3-raw.txt:[Caller 11, Robyn, F] Hi um I read the book quite a while ago and ...
# COME1-raw.txt:[Caller 23, Maureen, F] Yes good morning.
# NAT7-raw.txt:[Caller 12, Brian, M] Yeah.
# NAT8-raw.txt:[Caller 10, Brett, M] How're you going.


rawtext_dict['COME3-raw.txt'] = rawtext_dict['COME3-raw.txt'].replace('[Caller 9 Maureen, F]', '[Caller 9: Maureen, F]')

# $ grep -P '\[\S+ \d+ ' *
# COME3-raw.txt:[Caller 9 Maureen, F] Good morning Dr Graham.


rawtext_dict['COME3-raw.txt'] = rawtext_dict['COME3-raw.txt'].replace('[CE1]', '[E1]')

# $ grep CE1 *
# COME3-raw.txt:[CE1] If it did become serious <,> 


rawtext_dict['COME6-raw.txt'] = rawtext_dict['COME6-raw.txt'].replace('P1a', 'P1')
rawtext_dict['COME6-raw.txt'] = rawtext_dict['COME6-raw.txt'].replace('[P1b Paul Murray, M]', '[P1]').replace('P1b', 'P1')

# COME6 presenter encoding scheme was generally messed up.


rawtext_dict['COMNE4-raw.txt'] = rawtext_dict['COMNE4-raw.txt'].replace('[C14: Noelene, F]', '[Caller 14: Noelene, F]')

rawtext_dict['NAT1-raw.txt'] = rawtext_dict['NAT1-raw.txt'].replace('[C11]', '[C10]')

rawtext_dict['NAT5-raw.txt'] = rawtext_dict['NAT5-raw.txt'].replace('[E1] Thank you very much glad to', '[Expert 1: Brian Edgar, M] Thank you very much glad to')
rawtext_dict['NAT5-raw.txt'] = rawtext_dict['NAT5-raw.txt'].replace('<Expert 1: Brian Edgar, M thank you John>', '<E1 thank you John>')

rawtext_dict['COME3-raw.txt'] = rawtext_dict['COME3-raw.txt'].replace('[C4 Nah <E1 it ih thi> and because','[C4] Nah <E1 it ih thi> and because')
rawtext_dict['COME3-raw.txt'] = rawtext_dict['COME3-raw.txt'].replace('[E1 No no no <P1 we haven\'t had one> we','[E1] No no no <P1 we haven\'t had one> we')

# This is Marianna's only line - her speach is untranscribed: therefore I will skip her as a speaker
# print(rawtext_dict['NAT7-raw.txt'])
# rawtext_dict['NAT7-raw.txt'] = rawtext_dict['NAT7-raw.txt'].replace('{Caller 2: Marianna, F untranscribed overseas caller 04:32-07:18}','[Caller 2: Marianna, F untranscribed overseas caller 04:32-07:18]')

# print(rawtext_dict['COME3-raw.txt'])
rawtext_dict['COME3-raw.txt'] = rawtext_dict['COME3-raw.txt'].replace('of\ncholesterol excess cigarette smoking and of course a lack of exercise and uh central truncal obesity','of cholesterol excess cigarette smoking and of course a lack of exercise and uh central truncal obesity')

rawtext_dict['COME1-raw.txt'] = rawtext_dict['COME1-raw.txt'].replace('remember the name\n<inaudible> chrysanthemums','remember the name <inaudible> chrysanthemums')

# grep -P "Juicy" "COME6-raw.txt"
rawtext_dict['COME6-raw.txt'] = rawtext_dict['COME6-raw.txt'].replace('from a beautiful name called Juicy.\n\nJuicy.','from a beautiful name called Juicy. Juicy.')



# discovered while word and sentence tokenizing -- I was looking at all items within { } and < > so that I could remove them for the tokenization
# and saw that in this line, P1 is quoting Michelle, and the quote is inside { } -- I decided to remove the { } since it is still P1 speaking. I also added a period at the end.

# grep -P '{Hi Linda I have a Chinese lucky' 'COME1-raw.txt'

rawtext_dict['COME1-raw.txt'] = rawtext_dict['COME1-raw.txt'].replace("{Hi Linda I have a Chinese lucky bamboo that is looking very sick. It is kept in a narrow glass vase with glass rocks the roots appear to be orange and the smell of the water is disgusting <E1 laughs>. I have two shoots one is yellow half way up the stem the other appears to be okay. The leaves are wilting and yellow can I save my plant love Michelle}","Hi Linda I have a Chinese lucky bamboo that is looking very sick. It is kept in a narrow glass vase with glass rocks the roots appear to be orange and the smell of the water is disgusting <E1 laughs>. I have two shoots one is yellow half way up the stem the other appears to be okay. The leaves are wilting and yellow can I save my plant love Michelle.") 

In [14]:
# Looking at all {...} lines to find errors.
# The above cell accounts for the errors found here

for fid in files:
    rawtext = rawtext_dict[fid]
    rawlines = re.split(r'[\n\r]+', rawtext.strip())
    print(fid)
    for l in rawlines:
        if ']' in l:
            where = l.index(']')
            speaker = l[:where+1]
            utterance = l[where+2:]
            #print(speaker+' '+utterance)
        else: 
            print('***'+l+'*****')    # {program advert}. What to do with these? 
    print()

ABCE1-raw.txt
***{program advert}*****
***{program advert}*****
***{program advert 11:20-11:47}*****
***{program advert 19:53-20:23}*****
***{cut}*****
***{program advert}*****
***{program advert}*****
***{program advert 30:34-31:03}*****
***{program advert}*****
***{music}*****
***{music}*****
***{music}*****
***{music}*****
***{music}*****
***{program advert 37:19-37:32}*****
***{program advert}*****

ABCE2-raw.txt
***{program advert 14:53-15:24}*****
***{cut, program advert}*****
***{program advert}*****
***{program advert 24:28-24:58}*****
***{Untranscribed news bulletins 25:27-26:51}*****
***{program advert 34:22-34:33}*****
***{program advert 39:52-40:22}*****
***{program advert}*****

ABCE3-raw.txt
***{untranscribed book reading 21:44-22:12}*****

ABCE4-raw.txt

ABCNE1-raw.txt
***{program advert: 0:15:15- 0:15:46}*****
***{untranscribed poem, 25:51-26:24}*****
***{Ends 26:34}*****

ABCNE2-raw.txt
***{program advert 11:08-11:39}*****
***{program advert 18:25-18:56}*****
***{Ends 

## Speaker Information
### Creating Unique Speaker Ids

In [15]:
foo1 = "[Presenter 1: Simon Marnie, M]"
foo2 = "[Expert 1: Les, M]"
foo3 = "[Caller 10: Suzanne, F]"

def get_speaker(fid, sp):
    seg = fid[:-8]  # ABCE3-raw.txt -> ABCE3
    col = sp.index(':')
    
    if sp.startswith('[Presenter') :
        name = sp[col+2:-4].strip()
        return (seg+'-P'+sp[11:col], seg, 'P', sp[-2].upper(), name)
    elif sp.startswith('[Expert') :
        name = sp[col+2:-4].strip()
        return (seg+'-E'+sp[8:col], seg, 'E', sp[-2].upper(), name)
    elif sp.startswith('[Caller') :
        name = sp[col+2:-4].strip()
        return (seg+'-C'+sp[8:col], seg, 'C', sp[-2].upper(), name)
    else:
        return ()
get_speaker('ABCE3-raw.txt', foo1)
get_speaker('COMNE7-raw.txt', foo2)
get_speaker('ABCE3-raw.txt', foo3)

('ABCE3-P1', 'ABCE3', 'P', 'M', 'Simon Marnie')

('COMNE7-E1', 'COMNE7', 'E', 'M', 'Les')

('ABCE3-C10', 'ABCE3', 'C', 'F', 'Suzanne')

### Speaker Dictionary (speaker_dict):

- Key: unique speaker ID
- Values:
    - segment (ex: ABCE1, ABCE2, etc..)
    - role (P/C/E)
    - gender (F/M)
    - name
    
**NOTE: speaker_dict will be used to create the data frame speaker_df**    

In [16]:
# Creating a dictionary for each unique speaker

speaker_dict = {}

for fid in files: # includes NAT5-raw.txt
    rawtext = rawtext_dict[fid]
    rawlines = re.split(r'[\n\r]+', rawtext.strip())
    print(fid)
    for l in rawlines:
        if ']' in l:
            where = l.index(']')
            speaker = l[:where+1]
            if speaker.startswith('[Presenter') or speaker.startswith('[Expert') or speaker.startswith('[Caller'):
                #print(speaker)
                (uniq_id, seg, role, gender, name) = get_speaker(fid, speaker)
                print(uniq_id, seg, role, gender, name)
                speaker_dict[uniq_id] = (seg, role, gender, name)
        else: 
            #print('***'+l)    # {program advert}. What to do with these? 
            pass
    print()

ABCE1-raw.txt
ABCE1-P1 ABCE1 P M Simon Marnie
ABCE1-E1 ABCE1 E M Angus Stewart
ABCE1-C1 ABCE1 C F Suzanne
ABCE1-C2 ABCE1 C F Lisa
ABCE1-C3 ABCE1 C F Sally
ABCE1-C4 ABCE1 C M Danny
ABCE1-C5 ABCE1 C M Trevor
ABCE1-C6 ABCE1 C F Gillian
ABCE1-C7 ABCE1 C F Colleen
ABCE1-C8 ABCE1 C M Bernie
ABCE1-C9 ABCE1 C M William
ABCE1-E2 ABCE1 E F Jeanne Villani
ABCE1-C10 ABCE1 C F Beth
ABCE1-C11 ABCE1 C F Lynne
ABCE1-C12 ABCE1 C M Jack

ABCE2-raw.txt
ABCE2-P1 ABCE2 P M Simon Marnie
ABCE2-E1 ABCE2 E M Les
ABCE2-E2 ABCE2 E M Pete
ABCE2-C1 ABCE2 C F Julie
ABCE2-C2 ABCE2 C F Janelle
ABCE2-C3 ABCE2 C M Geoff
ABCE2-C4 ABCE2 C M Terry
ABCE2-E3 ABCE2 E M John Hall
ABCE2-C5 ABCE2 C F Judy
ABCE2-C6 ABCE2 C F Dawn
ABCE2-C7 ABCE2 C M Paul
ABCE2-C8 ABCE2 C F Linda
ABCE2-C9 ABCE2 C M Croydon
ABCE2-C10 ABCE2 C F Joan

ABCE3-raw.txt
ABCE3-P1 ABCE3 P F Lynne Haultain
ABCE3-P2 ABCE3 P F Jurate Sasnaitis
ABCE3-C1 ABCE3 C M Rick
ABCE3-C2 ABCE3 C F Sarah
ABCE3-C3 ABCE3 C F Liz
ABCE3-C4 ABCE3 C F Cathy
ABCE3-C5 ABCE3 C F Ki

In [17]:
# looking at each item in speaker_dict:

for s in sorted(speaker_dict):
    print (s, speaker_dict[s]) 

ABCE1-C1 ('ABCE1', 'C', 'F', 'Suzanne')
ABCE1-C10 ('ABCE1', 'C', 'F', 'Beth')
ABCE1-C11 ('ABCE1', 'C', 'F', 'Lynne')
ABCE1-C12 ('ABCE1', 'C', 'M', 'Jack')
ABCE1-C2 ('ABCE1', 'C', 'F', 'Lisa')
ABCE1-C3 ('ABCE1', 'C', 'F', 'Sally')
ABCE1-C4 ('ABCE1', 'C', 'M', 'Danny')
ABCE1-C5 ('ABCE1', 'C', 'M', 'Trevor')
ABCE1-C6 ('ABCE1', 'C', 'F', 'Gillian')
ABCE1-C7 ('ABCE1', 'C', 'F', 'Colleen')
ABCE1-C8 ('ABCE1', 'C', 'M', 'Bernie')
ABCE1-C9 ('ABCE1', 'C', 'M', 'William')
ABCE1-E1 ('ABCE1', 'E', 'M', 'Angus Stewart')
ABCE1-E2 ('ABCE1', 'E', 'F', 'Jeanne Villani')
ABCE1-P1 ('ABCE1', 'P', 'M', 'Simon Marnie')
ABCE2-C1 ('ABCE2', 'C', 'F', 'Julie')
ABCE2-C10 ('ABCE2', 'C', 'F', 'Joan')
ABCE2-C2 ('ABCE2', 'C', 'F', 'Janelle')
ABCE2-C3 ('ABCE2', 'C', 'M', 'Geoff')
ABCE2-C4 ('ABCE2', 'C', 'M', 'Terry')
ABCE2-C5 ('ABCE2', 'C', 'F', 'Judy')
ABCE2-C6 ('ABCE2', 'C', 'F', 'Dawn')
ABCE2-C7 ('ABCE2', 'C', 'M', 'Paul')
ABCE2-C8 ('ABCE2', 'C', 'F', 'Linda')
ABCE2-C9 ('ABCE2', 'C', 'M', 'Croydon')
ABCE2-E1 ('ABCE

### Speaker Data Frame (speaker_df):

In [18]:
speaker_df = pd.DataFrame(columns=['Segment', 'Speaker_Type', 'Gender', 'Name'], index=sorted(speaker_dict.keys()))
speaker_df.head()

,Segment,Speaker_Type,Gender,Name
ABCE1-C1,NaN,NaN,NaN,NaN
ABCE1-C10,NaN,NaN,NaN,NaN
ABCE1-C11,NaN,NaN,NaN,NaN
ABCE1-C12,NaN,NaN,NaN,NaN
ABCE1-C2,NaN,NaN,NaN,NaN


In [19]:
speaker_df['Segment'] = speaker_df.index.map(lambda x: speaker_dict[x][0])
speaker_df['Speaker_Type'] = speaker_df.index.map(lambda x: speaker_dict[x][1])
speaker_df['Gender'] = speaker_df.index.map(lambda x: speaker_dict[x][2])
speaker_df['Name'] = speaker_df.index.map(lambda x: speaker_dict[x][3])
speaker_df.head()

,Segment,Speaker_Type,Gender,Name
ABCE1-C1,ABCE1,C,F,Suzanne
ABCE1-C10,ABCE1,C,F,Beth
ABCE1-C11,ABCE1,C,F,Lynne
ABCE1-C12,ABCE1,C,M,Jack
ABCE1-C2,ABCE1,C,F,Lisa


In [20]:
speaker_df.tail()

,Segment,Speaker_Type,Gender,Name
NAT8-C5,NAT8,C,M,Theo
NAT8-C6,NAT8,C,F,Britney
NAT8-C8,NAT8,C,M,Peter
NAT8-C9,NAT8,C,M,Michael
NAT8-P1,NAT8,P,F,Gaby Brown


## Splitting the Texts by Line:

### List of Each Line of Text (art_list):

- unique speaker id for the line (ex: ABCE1-P1)
- utterance number of the *file*
- segment (ex: ABCE1)
- line of text

**NOTE: art_list will be used to create the data frame art_df**

In [21]:
# Creating a list of all the lines of text including the unique speaker id, utterance number (of the file), segment, and text line
    
art_list = []

for fid in files[:10]:
    rawtext = rawtext_dict[fid]
    rawlines = re.split(r'[\n\r]+', rawtext.strip())
    print(fid)
    utterance_num = 0
    for l in rawlines:
        if ']' in l:
            where = l.index(']')
            speaker = l[:where+1]
            utterance = l[where+2:]
            if speaker.startswith('[Presenter') or speaker.startswith('[Expert') or speaker.startswith('[Caller'):
                (uniq_id, seg, role, gender, name) = get_speaker(fid, speaker)
            else:
                print(l)
                seg = fid[:-8] # removes "-raw.txt"
                uniq_id = seg+'-'+speaker[1:-1]   # ex: ABCE1-C1
                (seg, role, gender, name) = speaker_dict[uniq_id]
            utterance_num += 1
            art_list.append((uniq_id, utterance_num, seg, utterance))
        else: 
            #print('***'+l)    # {program advert}. What to do with these? 
            pass

ABCE1-raw.txt
[P1] He's also known <E1 sounds reasonable> for his ability to open cosposting {composting} toilets so he can tell you anything worm farm problems certainly helped us and although I'm still confused about dry ingredients we might talk about that as well but eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two fine sunny day today top temperatures on the coast of twenty-seven inland thirty degrees Bowral enjoying twenty-seven and Katoomba twenty-five degrees currently around town on the coast it's seventeen that's four below <,> r Richmond and Bankstown are fifteen degrees Penrith sixteen Katoomba thirteen and Gosford twelve. One of the jewels in the open garden scheme crown is opening today and this is just a garden to envy how would you like <,> to have <,> a beautiful sandstone cottage nestled underneath a waterfall with a little pond and then a creek that runs through with thousands of water dragons so tame they come up and just <,> kiss you. Wo

[P2] Absolutely. I guess it just m it did actually wrap it up very <P1 neatly> very neatly <P1 yeah> and perhaps the relationship with the mother and father you know really if your wife had left for eight years perhaps you'd be a little angrier when she came back <P1 mm>. You know things like that but.
[P1] Yes and he was he was totally devoted  <P2 yes > the husband <P2 yes>. Let's talk about next month's book.
[P2] Oh goody.
[P1] Just in passing <P2 most excited> The Great Gatsby from F Scott Fitzgerald. Shouldn't be too hard to get hold of <P2 oh no> at all. And
[P2] Everywhere.
[P1] Everywhere in all the libraries and it's only about a hundred-and oh fifty odd pages so it's not gunna take you long and you've probably read it before anyway so it's just a refresher course between.
[P2] And wonderful films.
[P1] Here in April ih great films.
[P2] Three versions.
[P1] Are there three <P2 there are three now> I could only think of two.
[P2] There's the Alan Ladd version <P1 yeah> and th

[P2] Yes I I can j I can do that for you okay uh five and nine's fourteen's five <,> fourteen's five six five's are eleven. Okay that's good there since it's eleven it's a two actually uh the cycle you're both in.
[C14] Yes.
[P2] You're the uh very very compatible cycle <C14 oh okay>. Uh here at the moment because you've got the eleven <,> and you yourself have got the six so the six and the eleven like the eleven's a two they're very all to do with love family and ih that's exactly what it's all about there and passion. Mm mhm.
[C14] Oh okay. Actually I feel like I'm the only one that understands him really.
[P2] <laughs> Well I tell you what you're going through even though the six and the two cycle are not are are quite similar <,> and the whole thing is that the you're the next twelve months what you're both going through is all about beautiful love energies very very busy to do with the family and of course I mean that makes sense with the five kids. And also you've got uh lots an

In [22]:
# Examples from art_list:

art_list[0]
art_list[1]
art_list[2]
art_list[3]

('ABCE1-P1', 1, 'ABCE1', "Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation {propagation} Mr propagation. He's also known for his passion for natives and his love of o orchids am I right so far.")

('ABCE1-E1', 2, 'ABCE1', 'I guess yeah yeah <laughs>.')

('ABCE1-P1', 3, 'ABCE1', "He's also known <E1 sounds reasonable> for his ability to open cosposting {composting} toilets so he can tell you anything worm farm problems certainly helped us and although I'm still confused about dry ingredients we might talk about that as well but eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two fine sunny day today top temperatures on the coast of twenty-seven inland thirty degrees Bowral enjoying twenty-seven and Katoomba twenty-five degrees currently around town on the coast it's seventeen that's four below <,> r Richmond and Bankstown are fifteen degrees Penrith sixteen Katoomba thirteen and Gosford twelve. One of the jewels in the open garden scheme crown is opening today and this is just a garden to envy how would you like <,> to have <,> a beautiful sandstone cottage nestled underneath a waterfall with a little pond and then a creek that runs through with thousands of water dragons so tame they come up and just <,> kiss 

('ABCE1-E1', 4, 'ABCE1', 'Okay.')

### Text Lines Data Frame (art_df):

Content reminder:
- unique speaker ID
- utterance number
- segment
- line of text

**Note: The Data Frame will be indexed by unique speaker and the utterance number of the corresponding file.**

In [23]:
# To create a data frame from art_list, you must first create a numpy array

art_array = np.array(art_list)
art_array[:4]

array([['ABCE1-P1', '1', 'ABCE1',
        "Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation {propagation} Mr propagation. He's also known for his passion for natives and his love of o orchids am I right so far."],
       ['ABCE1-E1', '2', 'ABCE1', 'I guess yeah yeah <laughs>.'],
       ['ABCE1-P1', '3', 'ABCE1',
        "He's also known <E1 sounds reasonable> for his ability to open cosposting {composting} toilets so he can tell you anything worm farm problems certainly helped us and although I'm still confused about dry ingredients we might talk about that as well but eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two fine sunny day today top temperatures on the coas

In [24]:
art_df = pd.DataFrame(columns=['Speaker', 'Utterance_Number', 'Segment', 'Text'], data=art_array)
art_df.head()
art_df.tail()

,Speaker,Utterance_Number,Segment,Text
0,ABCE1-P1,1,ABCE1,Thanks for that John Hall now John Hall will b...
1,ABCE1-E1,2,ABCE1,I guess yeah yeah <laughs>.
2,ABCE1-P1,3,ABCE1,He's also known <E1 sounds reasonable> for his...
3,ABCE1-E1,4,ABCE1,Okay.
4,ABCE1-P1,5,ABCE1,Jeanne Villani does and we'll find out the sec...


,Speaker,Utterance_Number,Segment,Text
4434,COME5-P1,990,COME5,Jeez they're coming from everywhere tonight.
4435,COME5-P2,991,COME5,Bathurst yeah.
4436,COME5-P1,992,COME5,That's it now Deniloquin uh hi to you if you'r...
4437,COME5-P2,993,COME5,Day numbers <P1 day number> if you're into num...
4438,COME5-P1,994,COME5,Okay that's on the way. Sharina's Saturday Night.


In [25]:
# Sets the index to the speaker and utterance number

art_df = art_df.set_index(keys=["Speaker","Utterance_Number"])
art_df.head()
art_df.tail() 

,,Segment,Text
Speaker,Utterance_Number,,
ABCE1-P1,1,ABCE1,Thanks for that John Hall now John Hall will b...
ABCE1-E1,2,ABCE1,I guess yeah yeah <laughs>.
ABCE1-P1,3,ABCE1,He's also known <E1 sounds reasonable> for his...
ABCE1-E1,4,ABCE1,Okay.
ABCE1-P1,5,ABCE1,Jeanne Villani does and we'll find out the sec...


,,Segment,Text
Speaker,Utterance_Number,,
COME5-P1,990,COME5,Jeez they're coming from everywhere tonight.
COME5-P2,991,COME5,Bathurst yeah.
COME5-P1,992,COME5,That's it now Deniloquin uh hi to you if you'r...
COME5-P2,993,COME5,Day numbers <P1 day number> if you're into num...
COME5-P1,994,COME5,Okay that's on the way. Sharina's Saturday Night.


## Adding Number of Utterances per Speaker to the Speaker Data Frame:

In [26]:
# utterance dictionary
utt_dict=dict(art_df.groupby("Speaker").size())
for s in sorted(utt_dict):
    print (s, utt_dict[s]) 

ABCE1-C1 17
ABCE1-C10 17
ABCE1-C11 10
ABCE1-C12 12
ABCE1-C2 13
ABCE1-C3 6
ABCE1-C4 9
ABCE1-C5 11
ABCE1-C6 16
ABCE1-C7 18
ABCE1-C8 8
ABCE1-C9 13
ABCE1-E1 139
ABCE1-E2 27
ABCE1-P1 155
ABCE2-C1 16
ABCE2-C10 20
ABCE2-C2 16
ABCE2-C3 11
ABCE2-C4 16
ABCE2-C5 25
ABCE2-C6 20
ABCE2-C7 7
ABCE2-C8 23
ABCE2-C9 21
ABCE2-E1 192
ABCE2-E2 166
ABCE2-E3 6
ABCE2-P1 233
ABCE3-C1 5
ABCE3-C10 11
ABCE3-C11 6
ABCE3-C12 6
ABCE3-C2 11
ABCE3-C3 12
ABCE3-C4 10
ABCE3-C5 4
ABCE3-C6 4
ABCE3-C7 7
ABCE3-C8 9
ABCE3-C9 13
ABCE3-P1 129
ABCE3-P2 56
ABCE4-C1 6
ABCE4-C2 18
ABCE4-C3 13
ABCE4-C4 2
ABCE4-C5 5
ABCE4-C6 5
ABCE4-C7 6
ABCE4-E1 53
ABCE4-P1 23
ABCNE1-C1 2
ABCNE1-C2 13
ABCNE1-C3 7
ABCNE1-C4 2
ABCNE1-C5 4
ABCNE1-C6 1
ABCNE1-C7 7
ABCNE1-C8 3
ABCNE1-C9 3
ABCNE1-E1 74
ABCNE1-P1 64
ABCNE2-C1 8
ABCNE2-C2 7
ABCNE2-C3 9
ABCNE2-C4 16
ABCNE2-C5 15
ABCNE2-C6 13
ABCNE2-C7 14
ABCNE2-C8 9
ABCNE2-C9 12
ABCNE2-E1 104
ABCNE2-P1 58
COME1-C1 9
COME1-C10 12
COME1-C11 13
COME1-C12 14
COME1-C13 13
COME1-C14 8
COME1-C15 8
COME1-C16 13
COME1

In [27]:
# Reminder what speaker_df looks like:
speaker_df.head()

,Segment,Speaker_Type,Gender,Name
ABCE1-C1,ABCE1,C,F,Suzanne
ABCE1-C10,ABCE1,C,F,Beth
ABCE1-C11,ABCE1,C,F,Lynne
ABCE1-C12,ABCE1,C,M,Jack
ABCE1-C2,ABCE1,C,F,Lisa


In [28]:
utt_df = pd.DataFrame(columns=['Number_of_Utterances'], index=sorted(utt_dict.keys()))
utt_df['Number_of_Utterances'] = utt_df.index.map(lambda x: utt_dict[x])
utt_df.head()

,Number_of_Utterances
ABCE1-C1,17
ABCE1-C10,17
ABCE1-C11,10
ABCE1-C12,12
ABCE1-C2,13


In [29]:
# For this cell, I referenced Kyle Landin's project for his use of pd.merge

speaker_df=pd.merge(speaker_df,utt_df,right_index=True,left_index=True)

In [30]:
speaker_df.head()

,Segment,Speaker_Type,Gender,Name,Number_of_Utterances
ABCE1-C1,ABCE1,C,F,Suzanne,17
ABCE1-C10,ABCE1,C,F,Beth,17
ABCE1-C11,ABCE1,C,F,Lynne,10
ABCE1-C12,ABCE1,C,M,Jack,12
ABCE1-C2,ABCE1,C,F,Lisa,13


## Analysis of Number of Utterances by Speaker Type

### Creating Separate Presenter, Caller, and Expert Data Frames: 

In [31]:
# dataframe of presenters
P_df=speaker_df.loc[speaker_df["Speaker_Type"]=='P',:]
P_df.head()

# dataframe of callers
C_df=speaker_df.loc[speaker_df["Speaker_Type"]=='C',:]
C_df.head()

# dataframe of experts
E_df=speaker_df.loc[speaker_df["Speaker_Type"]=='E',:]
E_df.head()

,Segment,Speaker_Type,Gender,Name,Number_of_Utterances
ABCE1-P1,ABCE1,P,M,Simon Marnie,155
ABCE2-P1,ABCE2,P,M,Simon Marnie,233
ABCE3-P1,ABCE3,P,F,Lynne Haultain,129
ABCE3-P2,ABCE3,P,F,Jurate Sasnaitis,56
ABCE4-P1,ABCE4,P,F,Kelly Higgins-Devine,23


,Segment,Speaker_Type,Gender,Name,Number_of_Utterances
ABCE1-C1,ABCE1,C,F,Suzanne,17
ABCE1-C10,ABCE1,C,F,Beth,17
ABCE1-C11,ABCE1,C,F,Lynne,10
ABCE1-C12,ABCE1,C,M,Jack,12
ABCE1-C2,ABCE1,C,F,Lisa,13


,Segment,Speaker_Type,Gender,Name,Number_of_Utterances
ABCE1-E1,ABCE1,E,M,Angus Stewart,139
ABCE1-E2,ABCE1,E,F,Jeanne Villani,27
ABCE2-E1,ABCE2,E,M,Les,192
ABCE2-E2,ABCE2,E,M,Pete,166
ABCE2-E3,ABCE2,E,M,John Hall,6


**There are a lot more Callers than there are Presenters and Experts, but there are about equal numbers of Presenters and Experts**

In [32]:
print("Presenter Data Frame:")
P_df.info() # 12
print("\nCaller Data Frame:")
C_df.info() # 134
print("\nExpert Data Frame:")
E_df.info() # 16

Presenter Data Frame:
<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, ABCE1-P1 to COME5-P2
Data columns (total 5 columns):
Segment                 12 non-null object
Speaker_Type            12 non-null object
Gender                  12 non-null object
Name                    12 non-null object
Number_of_Utterances    12 non-null int64
dtypes: int64(1), object(4)
memory usage: 336.0+ bytes

Caller Data Frame:
<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, ABCE1-C1 to COME5-C9
Data columns (total 5 columns):
Segment                 134 non-null object
Speaker_Type            134 non-null object
Gender                  134 non-null object
Name                    134 non-null object
Number_of_Utterances    134 non-null int64
dtypes: int64(1), object(4)
memory usage: 3.7+ KB

Expert Data Frame:
<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, ABCE1-E1 to COME5-E2
Data columns (total 5 columns):
Segment                 16 non-null object
Speaker_Type            16

In [33]:
# Looking at the Presenter DF

P_df.max() # 491, Trevor Jackson
P_df.min() # 23, Jurate Sasnaitis
P_df.sum() # 1470 total lines
P_df.size # 60 cells
P_df.mean() # avg number of Utterances per Presenter = 122.5

Segment                          COME5
Speaker_Type                         P
Gender                               M
Name                    Trevor Jackson
Number_of_Utterances               491
dtype: object

Segment                            ABCE1
Speaker_Type                           P
Gender                                 F
Name                    Jurate Sasnaitis
Number_of_Utterances                  23
dtype: object

Segment                 ABCE1ABCE2ABCE3ABCE3ABCE4ABCNE1ABCNE2COME1COME...
Speaker_Type                                                 PPPPPPPPPPPP
Gender                                                       MMFFFMMMMMMF
Name                    Simon MarnieSimon MarnieLynne HaultainJurate S...
Number_of_Utterances                                                 1470
dtype: object

60

Number_of_Utterances    122.5
dtype: float64

In [34]:
# Comparing Presenter, Caller, and Expert Data Frames
P_df.describe()
C_df.describe()
E_df.describe()

,Number_of_Utterances
count,12.000000
mean,122.500000
std,129.384908
min,23.000000
25%,60.250000
50%,66.000000
75%,135.500000
max,491.000000


,Number_of_Utterances
count,134.000000
mean,11.231343
std,7.974503
min,1.000000
25%,6.000000
50%,10.000000
75%,14.000000
max,63.000000


,Number_of_Utterances
count,16.000000
mean,91.500000
std,86.817049
min,6.000000
25%,21.750000
50%,63.500000
75%,145.750000
max,294.000000


**Summary of Important Information by Role:**
    
Presenters: 
- 12 people
- average of 122.5 turns per person
- std: 129.38

Callers: 
- 134 people
- average of 11.23 turns per person
- std: 7.97

Experts:
- 16 people
- average of 91.5 turns per person
- std: 86.82

**Number of Utterances Analysis:**

Presenters speak the most throughout the Australian Radio Talkback Corpus. There are many callers in each show, but they do not speak for very long.

## Calculating Number of Sentences and Average Word Length by Speaker Type:

In [35]:
# art_list content reminder:
art_list[0]

for x in art_list[:2]:
    print(x[3])

('ABCE1-P1', 1, 'ABCE1', "Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation {propagation} Mr propagation. He's also known for his passion for natives and his love of o orchids am I right so far.")

Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation {propagation} Mr propagation. He's also known for his passion for natives and his love of o orchids am I right so far.
I guess yeah yeah <laughs>.


In [36]:
# How to replace part of a string using regular expressions (re.sub):

# help(re)
help(re.sub)

foo = "mary {had} a <little> lamb"

re.sub(r' <.*?>','',foo)
re.sub(r' {.*?}','',foo)
# re.findall(r'<.*?>',foo)
# re.findall(r'{.*?}',foo)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the match object and must return
    a replacement string to be used.



'mary {had} a lamb'

'mary a <little> lamb'

In [37]:
# Checking the contents of { } and < > before ignoring them
squir = []
for x in art_list:
    if re.findall(r'{.*?}',x[3]):
        squir.append([x[0],x[1],re.findall(r'{.*?}',x[3])]) # addressed issue with COME1-raw.txt line in the Data Cleaning Cell
print(squir)

carr = []
for x in art_list:
    if re.findall(r'<.*?>',x[3]):
        carr.append(re.findall(r'<.*?>',x[3])) # addressed issue with COME1-raw.txt line in the Data Cleaning Cell
print(carr)

[['ABCE1-P1', 1, ['{propagation}']], ['ABCE1-P1', 3, ['{composting}']], ['ABCE1-P1', 96, ['{foreign string}']], ['ABCE1-P1', 100, ['{foreign string}']], ['ABCE1-E1', 165, ['{break}']], ['ABCE1-P1', 188, ['{along}']], ['ABCE1-E2', 318, ['{rainforest}']], ['ABCE2-E1', 21, ['{Simon}', '{silence}']], ['ABCE2-E1', 124, ['{Delroy}']], ['ABCE2-E1', 311, ['{gunna}']], ['ABCE2-P1', 362, ['{every/any}']], ['ABCE2-P1', 366, ['{Piper}', '{peck of}']], ['ABCE2-P1', 491, ['{Illawong}']], ['ABCE2-E1', 658, ['{break}']], ['ABCE2-E1', 689, ['{break}']], ['ABCE2-E1', 701, ['{break}']], ['ABCE2-E2', 716, ['{break}']], ['ABCE2-E1', 719, ['{break}']], ['ABCE2-C10', 721, ['{break}']], ['ABCE2-E1', 755, ['{break}']], ['ABCE2-P1', 768, ['{break}']], ['ABCE4-E1', 1, ['{break}']], ['ABCE4-E1', 66, ['{cat/catbird imitation}']], ['ABCE4-E1', 74, ['{whistles catbird imitation}']], ['ABCNE1-C8', 154, ['{break}']], ['ABCNE1-E1', 167, ['{Oceania}', '{Oceania}', '{Oceania}', '{foliage}', '{Oceania}']], ['ABCNE1-E1', 1

### Trial Run: Word and Sentence Tokeninzation

In [38]:
# Trial run on the first two lines:
word_toks = []
sents = []

for x in art_list[:2]:
    line = x[3]
    print(line)
    
    line = re.sub(r' <.*?>','',line)
    line = re.sub(r' {.*?}','',line) # for now, removing all spelling corrections
    
    print(line)
    
    speaker=x[0]
    utt_num=x[1]
    
    # for now, including the speaker and utterance number of the file to make sure that word_toks and sents line up with art_df's indices
    word_toks.append([speaker,utt_num,nltk.word_tokenize(line)]) 
    sents.append([speaker,utt_num,nltk.sent_tokenize(line)])  
    
word_toks
sents

Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation {propagation} Mr propagation. He's also known for his passion for natives and his love of o orchids am I right so far.
Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation Mr propagation. He's also known for his passion for natives and his love of o orchids am I right so far.
I guess yeah yeah <laughs>.
I guess yeah yeah.


[['ABCE1-P1', 1, ['Thanks', 'for', 'that', 'John', 'Hall', 'now', 'John', 'Hall', 'will', 'be', 'listening', 'for', 'the', 'next', 'hour', "'cos", 'Angus', 'Stewart', 'is', 'here', 'to', 'take', 'your', 'calls', 'eight-triple-three-one-thousand', 'one-eight-hundred-eight-hundred-seven-oh-two', 'something', 'in', 'the', 'garden', 'that', "'s", 'causing', 'you', 'problems', 'give', 'us', 'a', 'call', 'right', 'now', 'and', 'Angus', 'can', 'I', 'mean', "y'know", 'he', 'is', 'known', 'in', 'the', 'trade', 'as', 'Mr', 'popergation', 'Mr', 'propagation', '.', 'He', "'s", 'also', 'known', 'for', 'his', 'passion', 'for', 'natives', 'and', 'his', 'love', 'of', 'o', 'orchids', 'am', 'I', 'right', 'so', 'far', '.']], ['ABCE1-E1', 2, ['I', 'guess', 'yeah', 'yeah', '.']]]

[['ABCE1-P1', 1, ["Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation Mr propagation.", "He's also known for his passion for natives and his love of o orchids am I right so far."]], ['ABCE1-E1', 2, ['I guess yeah yeah.']]]

### Word and Sentence Tokenization:

In [39]:
# Word and Sentence Tokenziation:
word_toks = []
sents = []
num_words = []
num_sents = []

for x in art_list:
    line = x[3]

    line = re.sub(r'<.*?>','',line)
    line = re.sub(r'{.*?}','',line) # for now, removing all spelling corrections
    
    word_toks_1 = nltk.word_tokenize(line)
    sents_1 = nltk.sent_tokenize(line)
    num_words_1 = len(word_toks_1)
    num_sents_1 = len(sents_1)
    
    word_toks.append(word_toks_1)
    sents.append(sents_1)    
    num_words.append(num_words_1)
    num_sents.append(num_sents_1)
    

word_toks[:5]
len(word_toks)

sents[:5]
len(sents)

[['Thanks', 'for', 'that', 'John', 'Hall', 'now', 'John', 'Hall', 'will', 'be', 'listening', 'for', 'the', 'next', 'hour', "'cos", 'Angus', 'Stewart', 'is', 'here', 'to', 'take', 'your', 'calls', 'eight-triple-three-one-thousand', 'one-eight-hundred-eight-hundred-seven-oh-two', 'something', 'in', 'the', 'garden', 'that', "'s", 'causing', 'you', 'problems', 'give', 'us', 'a', 'call', 'right', 'now', 'and', 'Angus', 'can', 'I', 'mean', "y'know", 'he', 'is', 'known', 'in', 'the', 'trade', 'as', 'Mr', 'popergation', 'Mr', 'propagation', '.', 'He', "'s", 'also', 'known', 'for', 'his', 'passion', 'for', 'natives', 'and', 'his', 'love', 'of', 'o', 'orchids', 'am', 'I', 'right', 'so', 'far', '.'], ['I', 'guess', 'yeah', 'yeah', '.'], ['He', "'s", 'also', 'known', 'for', 'his', 'ability', 'to', 'open', 'cosposting', 'toilets', 'so', 'he', 'can', 'tell', 'you', 'anything', 'worm', 'farm', 'problems', 'certainly', 'helped', 'us', 'and', 'although', 'I', "'m", 'still', 'confused', 'about', 'dry', 

4439

[["Thanks for that John Hall now John Hall will be listening for the next hour 'cos Angus Stewart is here to take your calls eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two something in the garden that's causing you problems give us a call right now and Angus can I mean y'know he is known in the trade as Mr popergation  Mr propagation.", "He's also known for his passion for natives and his love of o orchids am I right so far."], ['I guess yeah yeah .'], ["He's also known  for his ability to open cosposting  toilets so he can tell you anything worm farm problems certainly helped us and although I'm still confused about dry ingredients we might talk about that as well but eight-triple-three-one-thousand one-eight-hundred-eight-hundred-seven-oh-two fine sunny day today top temperatures on the coast of twenty-seven inland thirty degrees Bowral enjoying twenty-seven and Katoomba twenty-five degrees currently around town on the coast it's seventeen that's four be

4439

In [40]:
# art_df content reminder:
art_df.head()

,,Segment,Text
Speaker,Utterance_Number,,
ABCE1-P1,1,ABCE1,Thanks for that John Hall now John Hall will b...
ABCE1-E1,2,ABCE1,I guess yeah yeah <laughs>.
ABCE1-P1,3,ABCE1,He's also known <E1 sounds reasonable> for his...
ABCE1-E1,4,ABCE1,Okay.
ABCE1-P1,5,ABCE1,Jeanne Villani does and we'll find out the sec...


### Adding Word Tokens, Number of Words, Sentences and Number of Sentences to art_df:

In [41]:
art_df["Word_Toks"] = word_toks
art_df["Num_Words"] = num_words
art_df["Sents"] = sents
art_df["Num_Sents"] = num_sents
art_df.head()
art_df.tail()

,,Segment,Text,Word_Toks,Num_Words,Sents,Num_Sents
Speaker,Utterance_Number,,,,,,
ABCE1-P1,1,ABCE1,Thanks for that John Hall now John Hall will b...,"[Thanks, for, that, John, Hall, now, John, Hal...",80,[Thanks for that John Hall now John Hall will ...,2
ABCE1-E1,2,ABCE1,I guess yeah yeah <laughs>.,"[I, guess, yeah, yeah, .]",5,[I guess yeah yeah .],1
ABCE1-P1,3,ABCE1,He's also known <E1 sounds reasonable> for his...,"[He, 's, also, known, for, his, ability, to, o...",159,[He's also known for his ability to open cosp...,3
ABCE1-E1,4,ABCE1,Okay.,"[Okay, .]",2,[Okay.],1
ABCE1-P1,5,ABCE1,Jeanne Villani does and we'll find out the sec...,"[Jeanne, Villani, does, and, we, 'll, find, ou...",50,[Jeanne Villani does and we'll find out the se...,1


,,Segment,Text,Word_Toks,Num_Words,Sents,Num_Sents
Speaker,Utterance_Number,,,,,,
COME5-P1,990,COME5,Jeez they're coming from everywhere tonight.,"[Jeez, they, 're, coming, from, everywhere, to...",8,[Jeez they're coming from everywhere tonight.],1
COME5-P2,991,COME5,Bathurst yeah.,"[Bathurst, yeah, .]",3,[Bathurst yeah.],1
COME5-P1,992,COME5,That's it now Deniloquin uh hi to you if you'r...,"[That, 's, it, now, Deniloquin, uh, hi, to, yo...",51,[That's it now Deniloquin uh hi to you if you'...,1
COME5-P2,993,COME5,Day numbers <P1 day number> if you're into num...,"[Day, numbers, if, you, 're, into, numerology,...",10,"[Day numbers if you're into numerology., Mhm.]",2
COME5-P1,994,COME5,Okay that's on the way. Sharina's Saturday Night.,"[Okay, that, 's, on, the, way, ., Sharina, 's,...",12,"[Okay that's on the way., Sharina's Saturday N...",2


## *IN PROGRESS:* Back Channels 

In [42]:
# # Regular Expressions - Trial run with back channels: 

# bk_chnl_list = []
# other_speakers_list = []

# foo=art_list[:20]

# for x in foo:
    
#     line=x[3]
#     speaker=x[0]
#     segment=x[2]
    
#     print(speaker)
#     print(line)
#     print(segment)
    
#     re.findall(r'<.*?>',line)
#     bk_chnl_list.append(re.findall(r'<.*?>',line)) # every instance of a <...> 
#     # need to remove <inaudible> and <,>

#     print("speaker:")
#     found = re.findall(r'<[PCE]+[0-9]+',line) # includes < 
    
#     other_speakers_list.append(re.findall(r'<[PCE]+[0-9]+',line))
   
#     for b in other_speakers_list[-1]:
#     # a for loop to remove '<' from the other speakers list and replace with the segment (creating unique speaker ID):
#         b_index=other_speakers_list[-1].index(b)
#         if b.startswith('<')==True:
#             other_speakers_list[-1][b_index]=b.replace('<',segment+"-")

# bk_chnl_list
# other_speakers_list